In [ ]:
pip install boto3


## For using External stage as intermediate staging medium 

In [6]:
import cx_Oracle
import boto3
import json
import snowflake.connector

 

# Oracle Database Connection

oracle_connection = cx_Oracle.connect('system/admin@localhost:1521/XE')

oracle_cursor = oracle_connection.cursor()

 

#Snowflake Connection

with open('Credentials.json') as config_file:
    config = json.load(config_file)


snowflake_connection = snowflake.connector.connect(

    user = config["USER"],

    password = config["PASSWORD"],

    account = config["ACCOUNT"],

    #warehouse= config["WAREHOUSE"],

    database= config["DATABASE"],

    schema= config["SCHEMA"]

)

 

# Export data from Oracle to CSV

oracle_cursor.execute('select * from employees')

data = oracle_cursor.fetchall()

 

# Write data to a CSV file

with open('data.csv', 'w') as csv_file:

    for row in data:

        csv_file.write(','.join(map(str, row)) + '\n')

 

# Upload CSV to S3

s3_client = boto3.client('s3', aws_access_key_id = config["aws_access_key_id"], aws_secret_access_key = config["aws_secret_access_key"])

s3_client.upload_file('data.csv', 'etlsnowflake21', 'data.csv')

print("File Successfully uploaded on S3 Bucket !")

 

# Loading CSV data from S3 into Snowflake

snowflake_cursor = snowflake_connection.cursor()

STAGE_NAME="@demo_db.public.ext_csv_stage"

snowflake_cursor.execute("COPY INTO DEMO_DB.PUBLIC.EMPLOYEES FROM {} FILE_FORMAT=(TYPE=CSV SKIP_HEADER=0)".format(STAGE_NAME))

print("File Successfully loaded in Snowflake table !")



# Closing my connections

oracle_cursor.close()

oracle_connection.close()

snowflake_cursor.close()

snowflake_connection.close()

File Successfully uploaded on S3 Bucket !
File Successfully loaded in Snowflake table !


## For using External stage as intermediate staging medium 

In [22]:
import cx_Oracle
import snowflake.connector
import json
 

# Oracle Database Connection

oracle_connection = cx_Oracle.connect('system/admin@localhost:1521/XE')

oracle_cursor = oracle_connection.cursor()

 

with open('Credentials.json') as config_file:
    config = json.load(config_file)


# Snowflake Connection

snowflake_connection = snowflake.connector.connect(

    user = config["USER"],

    password = config["PASSWORD"],

    account = config["ACCOUNT"],

    #warehouse= config["WAREHOUSE"],

    database= config["DATABASE"],

    schema= config["SCHEMA"]

)

 

# Export data from Oracle to CSV

oracle_cursor.execute('SELECT * FROM employees')

data = oracle_cursor.fetchall()

 

# Write data to a CSV file

with open('data.csv', 'w') as csv_file:

    for row in data:

        csv_file.write(','.join(map(str, row)) + '\n')

 

# Define Snowflake stage name

snowflake_stage = "demo_db.public.stg1"

 

# Copy data from the local CSV file to Snowflake stage

snowflake_cursor = snowflake_connection.cursor()

snowflake_cursor.execute(f"PUT file://data.csv @{snowflake_stage}")

 

# Load data from the stage into Snowflake table

snowflake_cursor.execute(f"COPY INTO employees FROM @{snowflake_stage}/data.csv FILE_FORMAT=(TYPE=CSV SKIP_HEADER=0)")

 

# Close database connections

oracle_cursor.close()

oracle_connection.close()

snowflake_cursor.close()

snowflake_connection.close()